In [13]:
import pennylane as qml
from pennylane import numpy as np

In [14]:
n_wires = 4
dev = qml.device("default.qubit", wires = n_wires)

@qml.qnode(dev)
def entangler_circuit(weights):
  qml.BasicEntanglerLayers(weights, wires = range(n_wires))
  return qml.expval(qml.PauliZ(0))

print(qml.draw(entangler_circuit, level = "device")([[0.1,0.2,0.3,0.4],[0.5,0.6,0.7, 0.8]]))

0: ──RX(0.10)─╭●───────╭X──RX(0.50)─╭●───────╭X─┤  <Z>
1: ──RX(0.20)─╰X─╭●────│───RX(0.60)─╰X─╭●────│──┤     
2: ──RX(0.30)────╰X─╭●─│───RX(0.70)────╰X─╭●─│──┤     
3: ──RX(0.40)───────╰X─╰●──RX(0.80)───────╰X─╰●─┤     


We can calculate the gradient of the function $F: \mathbb{R}^4 \to \mathbb{R}$ defined by $F(\theta_0, \dots, \theta_3) := \langle Z_0 \rangle$.

In [15]:
n_wires = 4
dev = qml.device("default.qubit", wires = n_wires)

@qml.qnode(dev)
def entangler(weights):
    qml.BasicEntanglerLayers(weights, wires = range(n_wires))
    return qml.expval(qml.PauliZ(0))

test_weights = np.array([[0.1,0.2,0.3,0.4]], requires_grad = True)
print(qml.jacobian(entangler)(test_weights))


[[ 1.38777878e-17 -1.74813749e-01 -2.66766415e-01 -3.64609810e-01]]


Use parameter shift rule to calculate the jacobian

In [16]:
n_wires = 4
dev = qml.device("default.qubit", wires = n_wires)

@qml.qnode(dev, interface="autograd", diff_method="parameter-shift")
def entangler(weights):
    qml.BasicEntanglerLayers(weights, wires = range(n_wires))
    return qml.expval(qml.PauliZ(0))

test_weights = np.array([[0.1,0.2,0.3,0.4]], requires_grad = True)
print(qml.jacobian(entangler)(test_weights))

[[ 1.52785233e-17 -1.74813749e-01 -2.66766415e-01 -3.64609810e-01]]


We want some circuit parameters to be differentiated, while others should remain fixed. This is the reason why the requires_grad option is important. For example, let's consider two Basic Entangler layers and differentiate only with respect to the parameters diff_weights in the first layer. The parameters fixed_weights in the second layer remain constant. Let's also make sure that we use the parameter-shift rule.

In [22]:
n_wires = 4
dev = qml.device("default.qubit", wires=n_wires)

@qml.qnode(dev, interface="autograd", diff_method="parameter-shift")
def entangler_fixed(diff_weights, fixed_weights):
    qml.BasicEntanglerLayers(diff_weights, wires=range(n_wires))
    qml.BasicEntanglerLayers(fixed_weights, wires=range(n_wires))
    return qml.expval(qml.PauliZ(0))

test_diff_weights = np.array([[0.5,0.1,-0.4,0.6]], requires_grad = True)
test_fixed_weights = np.array([[0.1,0.2,0.3,0.4]], requires_grad = False)

print(qml.jacobian(entangler_fixed)(test_diff_weights, test_fixed_weights))
print(qml.draw(entangler_fixed, level="device")(test_diff_weights, test_fixed_weights))

[[-3.06476461e-01  6.16087156e-02 -1.58799709e-17 -4.13038527e-01]]
0: ──RX(0.50)──╭●───────╭X──RX(0.10)─╭●───────╭X─┤  <Z>
1: ──RX(0.10)──╰X─╭●────│───RX(0.20)─╰X─╭●────│──┤     
2: ──RX(-0.40)────╰X─╭●─│───RX(0.30)────╰X─╭●─│──┤     
3: ──RX(0.60)────────╰X─╰●──RX(0.40)───────╰X─╰●─┤     


We can also calculate Jacobian matrix of a circuit. For example, if the output of a circuit with $k$ wires are the measurement probabilities, the output can be a real-valued vector $(F_0, \dots, F_{m-1})$ with $m = 2^k$ components. If the circuit depends on $n$ gate parameters $(\theta_0, \dots, \theta_{n-1})$, then the circuit can be interpreted as a function
$$
F: \mathbb{R}^n \longrightarrow \mathbb{R}^m
$$
where $(\theta_0, \dots, \theta_{n-1}) \mapsto (F_0, \dots, F_{m-1})$.

In [25]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def vector_valued_circuit(params):
    qml.RX(params[0], wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(params[1], wires=0)
    return qml.probs(wires=[0,1])

sample_params = np.array([0.1,0.2], requires_grad = True)
print(qml.jacobian(vector_valued_circuit)(sample_params)) # we expect 4x2 matrix, since there are 2 parameters and 4 computational basis outcome probabilities.


[[-0.0494192  -0.09908654]
 [ 0.00049751  0.00024813]
 [-0.00049751  0.09908654]
 [ 0.0494192  -0.00024813]]


We can findn the Hessian of a function $F: \mathbb{R}^n \to \mathbb{R}$ as the Jacobian of the gradient of $F$.

In [26]:
dev = qml.device("default.qubit", wires = 2)

@qml.qnode(dev, diff_method = "parameter-shift", max_diff = 2)
def scalar_valued_circuit(params):
  qml.RX(params[0], wires = 0)
  qml.CNOT(wires=[0,1])
  qml.RY(params[1], wires = 0)
  return qml.expval(qml.PauliZ(0))

test_params = np.array([0.7,0.3], requires_grad = True)
qml.jacobian(qml.jacobian(scalar_valued_circuit))(test_params)

array([[-0.73068165,  0.19037934],
       [ 0.19037934, -0.73068165]])

Optimizing circuits

We would like to know what is the minimum expectation value that the circuit output $\langle Z_0 \rangle$ can have.

In [27]:
dev = qml.device("default.qubit", wires = 2)

@qml.qnode(dev, diff_method = "parameter-shift")
def scalar_valued_circuit(params):
    qml.RX(params[0], wires = 0)
    qml.CNOT(wires=[0,1])
    qml.RY(params[1], wires = 0)
    return qml.expval(qml.PauliZ(0))

In [28]:
def optimize(cost_function, init_params, *steps):
    opt = qml.GradientDescentOptimizer(stepsize=0.4)
    steps = 100
    params = init_params
    
    for i in range(steps):
        params = opt.step(cost_function, params)
    
    return params, cost_function(params)

In [31]:
initial_parameters = np.array([0.7,0.3], requires_grad = True)
print(optimize(scalar_valued_circuit, initial_parameters, 100))

(tensor([3.14159265e+00, 8.76940135e-17], requires_grad=True), array(-1.))


In [36]:
dev = qml.device("default.qubit", wires = 4)

@qml.qnode(dev)
def strong_entangler(params):
    """
    Applies Strongly Entangling Layers to the default initial state
    Args:
    - weights (np.ndarray): The weights argument for qml.StronglyEntanglingLayers
    Returns:
    - (np.tensor): <Z0>
    """

    ####################
    ###YOUR CODE HERE###
    ####################
    qml.StronglyEntanglingLayers(weights=params, wires=range(4))
    
    return qml.expval(qml.PauliZ(0))

test_weights = np.array([[[0.1, 0.2, 0.3],
                          [0.4, 0.5, 0.6],
                          [0.7, 0.8, 0.9],
                          [1.0, 1.1, 1.2]]])

print("The output of your circuit with these weights is: ", strong_entangler(test_weights))


The output of your circuit with these weights is:  0.2773366786368507
